# Tabla de Contenido

* [1 Cargar los datos](#1)
* [2 Separar los datos en dos conjuntos](#2)
* [3 Evaluacion de los modelos](#3)
* [4 Validacion del modelo](#4)
* [5 Conclusion](#5)

## Cargar los datos

In [3]:
# cargamos las librerías

import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression



In [4]:
# cargamos el dataframe a usar

df = pd.read_csv('/datasets/users_behavior.csv')

In [5]:
# visualizamos el dataframe

df

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


In [6]:
# observamos las generalidades del dataframe

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


**Se procede a trabajar ya que no observamos ninguna anomalía en los datos**

## Separar los datos en dos conjuntos

In [7]:
# definimos los datos en conjuntos de entrenamiento y prueba

df_train, df_test = train_test_split(df, test_size = 0.20, random_state = 12345)

In [8]:
# separamos los datos

features_train = df_train.drop(['is_ultra'], axis = 1)

features_test = df_test.drop(['is_ultra'], axis = 1)

target_train = df_train['is_ultra']

target_test = df_test['is_ultra']

## Evaluación de los modelos

**Acá, procederemos a evaluar los tres modelos: de arbol de decision, de bosque aleatorio y de regresión logística**

In [9]:
# Evaluamos el modelo con el arbol de decision con datos de entrenamiento

#best_model = None
#best_result = 0

#for depth in range(1, 6):
   # model = DecisionTreeClassifier(random_state = 12345, max_depth = depth)
   # model.fit(features_train, target_train)
   # predictions = model.predict(features_train)
   # result = accuracy_score(target_train, predictions)
    
   # if result > best_result:
    #    best_model = model
    #    best_result = result
    
# print('EXACTITUD DEL MEJOR MODELO EN EL ENTRENAMIENTO:', best_result)

In [10]:
# Evaluamos el modelo con el arbol de decision con datos de prueba 

best_model = None
best_result = 0

for depth in range(1, 6):
    model = DecisionTreeClassifier(random_state = 12345, max_depth = depth)
    model.fit(features_train, target_train)
    predictions = model.predict(features_test)
    result = accuracy_score(target_test, predictions)
    
    if result > best_result:
        best_model = model
        best_result = result
    
print('EXACTITUD DEL MEJOR MODELO EN EL ENTRENAMIENTO CON DATOS DE PRUEBA:', best_result)

EXACTITUD DEL MEJOR MODELO EN EL ENTRENAMIENTO CON DATOS DE PRUEBA: 0.7853810264385692


**Se puede notar que hay un sobreajuste en la evaluación del 3%**

In [18]:
# Evaluamos el modelo de bosque con datos de entrenamiento

#best_score = 0 
#best_est = 0

#for est in range(1, 11):
#    model = RandomForestClassifier(random_state = 12345, n_estimators = est)
#    model.fit(features_train, target_train)
#    score = model.score(features_train, target_train)
#    if score > best_score:
#        best_score = score
#        best_est = est
        
#print("PRECISION DEL MEJOR MODEL DEL ENTRENAMIENTO: (n_estimators = {}): {}".format(best_est, best_score))        

PRECISION DEL MEJOR MODEL DEL ENTRENAMIENTO: (n_estimators = 9): 0.9782185919875535


In [11]:
# Evaluamos el modelo de bosque con datos de prueba

best_score = 0 
best_est = 0

for est in range(1, 11):
    model = RandomForestClassifier(random_state = 12345, n_estimators = est)
    model.fit(features_train, target_train)
    score = model.score(features_test, target_test)
    
    if score > best_score:
        best_score = score
        best_est = est
        
print("PRECISION DEL MEJOR MODEL DEL ENTRENAMIENTO CON LOS DATOS DE PRUEBA: (n_estimators = {}): {}".format(best_est, best_score))     

PRECISION DEL MEJOR MODEL DEL ENTRENAMIENTO CON LOS DATOS DE PRUEBA: (n_estimators = 4): 0.7744945567651633


**Esta evaluación muestra un sobreajuste del 20%**

In [12]:
# Evaluacion del modelo de regresion logistica

model = LogisticRegression(random_state = 12345, solver = 'liblinear')
model.fit(features_train, target_train)
score_train = model.score(features_train, target_train)
score_test = model.score(features_test, target_test)

print('PRECISION DEL MODELO DE REGRESION LOGISTICA EN EL ENTRENAMIENTO:', score_train)
print('PRECISION DEL MODELO DE REGRESION LOGISTICA EN LA PRUEBA: ', score_test)

PRECISION DEL MODELO DE REGRESION LOGISTICA EN EL ENTRENAMIENTO: 0.7016725009723843
PRECISION DEL MODELO DE REGRESION LOGISTICA EN LA PRUEBA:  0.702954898911353


**Se pudo observar que no presenta sobreajuste determinante además que se que ambos modelos se muestran por debajo de 0.75**

**Se determinó que el mejor modelo que mejor resultados nos da es el del arbol de decision, ya que supera los 0.75**

In [40]:
# Evaluacion del modelo de cordura (Model Sanity Check)

from sklearn.dummy import DummyClassifier

model = DummyClassifier(random_state = 42)
model.fit(features_test, target_test)
predictions = dummy_model.predict(features_test)
score_train= model.score(features_train, target_train)
score_test = model.score(features_test, target_test)


print('RESULTADO DE ASEVERACION DEL ESTUDIO DEL CONJUNTO DE ENTRENAMIENTO:', score_train)
print('RESULTADO DE ASEVERACION DEL ESTUDIO DEL CONJUNTO DE PRUEBA:', score_test)





RESULTADO DE ASEVERACION DEL ESTUDIO DEL CONJUNTO DE ENTRENAMIENTO: 0.6931155192532089
RESULTADO DE ASEVERACION DEL ESTUDIO DEL CONJUNTO DE PRUEBA: 0.6951788491446346


Se procedio a colocar la evaluacion del modelo de cordura, que es las que nos indica si el resultado de un calculo o una aseveración puede ser posible. Se importo la libreria correspondiente y los resultados fueron por debajo del 0.75.

## Validación del modelo

In [23]:
# definimos los datos en conjuntos de entrenamiento y validación

df_train, df_valid = train_test_split(df, test_size = 0.25, random_state = 12345)

In [24]:
# separamos los datos 

features_train = df_train.drop(['is_ultra'], axis = 1)

features_valid = df_valid.drop(['is_ultra'], axis = 1)

target_train = df_train['is_ultra']

target_valid = df_valid['is_ultra']

In [32]:
# Evaluamos el modelo con el arbol de decision con datos de validación

best_model = None
best_result = 0

for depth in range(1, 6):
    model = DecisionTreeClassifier(random_state = 12345, max_depth = depth)
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    result = accuracy_score(target_valid, predictions)
    
    if result > best_result:
        best_model = model
        best_result = result
    
print('EXACTITUD DEL MEJOR MODELO CON EL CONJUNTO DE VALIDACION:', best_result)

EXACTITUD DEL MEJOR MODELO CON EL CONJUNTO DE VALIDACION: 0.7885572139303483


<div class="alert alert-info">
<h2> Comentarios del estudiante</h2>
Al validar, podemos notar que la exactitud es de 0.7885 mientras que anteriormente habiamos obtenido 0.7853, una diferencia muy minima.
</div>

## Conclusion

* En el apartado de evaluacion de modelos, se puede observar que el modelo que da mejores resultados es el del árbol de decisión. ¿Por qué? Debido a que muestra una exactitud mayor a 0.75 (muestra 0.7885). 

* Los otros dos modelos, el del bosque aleatorio y de regresión logística mostraron datos que no se adecuan a lo que se busca, como una diferencia entre prueba y entrenamiento superior al 20% y unos resultados subajustados, por debajo de 0.75, respectivamente.

* Se recomienda utilizar el modelo del arbol de decisión, ya que los datos de prueba son similares a los datos de validación, la diferencia es muy minima.

<div class="alert alert-danger">
<h2> Comentarios del revisor </h2>

    
Parece que hay código duplicados, con celdas correctas y otras no. Revisa las celdas que vas a dejar al final, recuerda que adicionalmente el conjunto debe dividirse en datos de entrenamiento, uno de validación y uno de prueba.Y si tienes la oportunidad implementar el modelo de cordura, seguro no tendrás problemas con las correcciones. Saludos!</div>

<div class="alert alert-success">
<h2> Comentarios del revisor V2</h2>

Muy buen trabajo con todas las correcciones Douglas! Felicitaciones por aprobar todos los puntos del proyecto!
</div>